In [1]:
%reload_ext autoreload
%aimport -pandas, -numpy
%autoreload 2

import pandas as pd
from dmyplant2 import cred, MyPlant, Engine
import dplayFSM2
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [2]:
e=Engine.from_sn(mp,1486174)
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
e['IB Site Name'] + ' ' + e['Engine ID'], modes, success

('Forsa Hartmoor M04', ['???', 'OFF', 'MANUAL', 'AUTO'], [True, False])

In [11]:
e=Engine.from_sn(mp,1486174)
fsm = dplayFSM2.demoFSM(e, p_from='2020-01-01', p_to='2022-02-01', successtime=600)
#fsm = dplayFSM2.demoFSM(e, p_to='2022-03-10', successtime=600)
fsm.run1(enforce=True) # run Finite State Machine

In [12]:
rda = fsm.starts
rda[['no','success','mode','starttime','endtime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','maxload','ramprate','targetoperation','rampdown','coolrun','runout','count_alarms', 'count_warnings']].round(2).fillna('-')

,no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,maxload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,0,False,MANUAL,305.44,-,-,-,-,-,305.44,-,-,-,-,-,-,0,0
1,1,False,MANUAL,292.93,3.93,26.24,-,-,-,323.10,-,-,-,-,-,-,0,1
2,2,False,MANUAL,64.85,-,-,-,-,-,64.85,-,-,-,-,-,-,1,0
3,3,False,MANUAL,44.80,-,-,-,-,-,44.80,-,-,-,-,-,-,1,0
4,4,False,MANUAL,175.84,3.52,19.97,40.59,-,-,239.93,-,-,-,-,-,-,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,True,AUTO,97.94,3.73,23.21,4.44,37.44,110.37,277.13,-,-,3622.06,127.61,9.98,49.12,0,0
110,110,True,AUTO,100.07,3.73,19.99,4.34,1.41,0.0,129.54,-,-,1822.54,127.51,9.98,48.76,0,0
111,111,True,AUTO,100.39,3.62,25.82,4.54,4.14,137.89,276.40,-,-,13222.3,127.37,10.01,48.77,0,0
112,112,True,AUTO,97.09,3.64,22.92,4.64,4.42,137.33,270.04,-,-,5422.72,127.36,9.99,48.82,0,1


In [13]:
rdc = fsm.stops
rdc[['no','mode','starttime','endtime','count_alarms','count_warnings']].round(2).fillna('-')

,no,mode,starttime,endtime,count_alarms,count_warnings
0,0,???,2021-06-15 09:54:30.254000128,2021-11-17 16:17:47.176999936,97.0,106.0
1,1,AUTO,2021-11-17 16:17:47.176999936,2021-11-17 17:45:19.983000064,1.0,0.0
2,2,MANUAL,2021-11-17 17:50:16.849999872,2021-11-17 17:55:23.932000000,2.0,0.0
3,3,MANUAL,2021-11-17 17:55:23.932000000,2021-11-17 17:57:22.967000064,0.0,0.0
4,4,MANUAL,2021-11-17 17:57:22.967000064,2021-11-17 18:00:02.241999872,0.0,0.0
...,...,...,...,...,...,...
110,110,AUTO,2022-01-26 08:32:20.913999872,2022-01-26 09:55:04.060000000,0.0,0.0
111,111,AUTO,2022-01-26 10:32:20.484000000,2022-01-27 16:45:05.318000128,0.0,0.0
112,112,AUTO,2022-01-27 20:32:21.395000064,2022-01-28 07:25:04.752000000,0.0,0.0
113,113,AUTO,2022-01-28 09:02:21.027000064,2022-01-31 16:55:08.587000064,0.0,0.0


In [8]:
fsm._runlogdetail

['=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 600 15.06.2021 09:54:30  9007 Hourly message',
 '=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 600 15.06.2021 09:54:34  1262 Demand oil pump (DC) off',
 '=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 600 15.06.2021 09:54:34  1260 Demand oil pump off',
 '=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 600 15.06.2021 09:56:11  1239 Group alarm - shut down',
 '=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 800 15.06.2021 09:56:11  1140 Module interface panel circuit breaker tripped',
 '=    init               2021-06-15 09:54:30.254000128 -> standstill        2021-06-15 09:54:30.254000128 600 15.06.2021 10:04:34  1282 Demand engine cooling water preheatin

In [7]:
fsm.starts.iloc[149].to_dict()

{'run2': False,
 'no': 149,
 'success': True,
 'mode': 'AUTO',
 'starttime': Timestamp('2022-03-04 19:43:12.007000064'),
 'endtime': Timestamp('2022-03-05 12:43:12.756999936'),
 'cumstarttime': 133.451997,
 'timing': {'start_startpreparation': Timestamp('2022-03-05 10:35:05.863000064'),
  'end_startpreparation': Timestamp('2022-03-05 10:36:43.496000'),
  'start_starter': Timestamp('2022-03-05 10:36:43.496000'),
  'end_starter': Timestamp('2022-03-05 10:36:47.736999936'),
  'start_speedup': Timestamp('2022-03-05 10:36:47.736999936'),
  'end_speedup': Timestamp('2022-03-05 10:37:10.736000'),
  'start_idle': Timestamp('2022-03-05 10:37:10.736000'),
  'end_idle': Timestamp('2022-03-05 10:37:15.182000128'),
  'start_synchronize': Timestamp('2022-03-05 10:37:15.182000128'),
  'end_synchronize': Timestamp('2022-03-05 10:37:19.315000064'),
  'start_loadramp': Timestamp('2022-03-05 10:39:42.627000064'),
  'end_loadramp': Timestamp('2022-03-05 10:39:42.627000064'),
  'start_targetoperation': Tim